# 这是关于Materials Project数据的获取

现在MP的API接口MPRester在mp_api里面，使用前先安装：pip install mp_api


以下脚本搜索Si单质的各个相，并保存为cif文件


In [1]:
from mp_api.client import MPRester

D:\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


我的Materials Project的api密钥为: igRHy7zYOKzWD18jY76XtjbwEpRl6SoH

In [ ]:
API_KEY = "igRHy7zYOKzWD18jY76XtjbwEpRl6SoH"

mpr = MPRester(API_KEY)#创建MPRester对象进行认证

with MPRester(API_KEY) as mpr:
    #docs = mpr.summary.search(material_ids=['mp-546794'])
    docs = mpr.summary.search(elements=['Si'],fields=['material_id','structure','energy_above_hull','theoretical'])#fields即需要下载的信息

#elements=['Si']表示包含Si的各种晶体，可以有其他元素
#chemsys='Si-O'表示仅包含Si和O的晶体
# from emmet.core.summary import HasProps
#has_props=[HasProps.dielectric]
#band_gap=(3,None)  #能带带隙大于3eV
#energy_above_hull=(0,0)

#mpr.get_bandstructure_by_material_id("mp-149") #能带
#mpr.get_dos_by_material_id("mp-149")           #态密度
#mpr.get_phonon_bandstructure_by_material_id("mp-149")  #声子谱
#mpr.get_phonon_dos_by_material_id("mp-149")            #声子态密度
#mpr.xas.search_xas_docs(formula = "TiO2",absorbing_element = 'Ti',edge = Edge.K)  #XAS
#mpr.get_charge_density_from_material_id("mp-149")   #电荷密度
mpr.thermo.get_phase_diagram_from_chemsys(chemsys="Li-Fe-O", thermo_type="GGA_GGA+U_R2SCAN")  #相图

In [98]:

for idoc in docs[:10]:  # 保存前10个结构到cif文件
    idoc.structure.to(stidoc.material_id+".cif")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12300\94081137.py:7: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(elements=['Si'],fields=['material_id','structure','energy_above_hull','theoretical'])#fields即需要下载的信息
Retrieving SummaryDoc documents: 100%|██████████| 12673/12673 [01:52<00:00, 112.67it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12300\94081137.py:22: DeprecationWarning: Accessing thermo data through MPRester.thermo is deprecated. Please use MPRester.materials.thermo instead.
  mpr.thermo.get_phase_diagram_from_chemsys(chemsys="Li-Fe-O", thermo_type="GGA_GGA+U_R2SCAN")  #相图
Retrieving ThermoDoc documents: 100%|██████████| 1/1 [00:00<00:00, 1002.46it/s]


# STEP1:获取material_id

In [18]:
import json
from tqdm import tqdm

from mp_api.client import MPRester
import pickle

API_KEY = "igRHy7zYOKzWD18jY76XtjbwEpRl6SoH"
mpr = MPRester(API_KEY)
# 测试获取元素Si的id
element_lst = ['Si']
entries = mpr.materials.summary.search(elements = element_lst, fields = ['material_id'])


Retrieving SummaryDoc documents: 100%|██████████| 12673/12673 [00:03<00:00, 3731.35it/s]


可以看到获取的信息类型为list，每个元素为MPDataDoc类，想要获取他的id需要使用.material_id

In [20]:
print(type(entries))
print(entries[0])
print(entries[0].material_id)

<class 'list'>
MPDataDoc<SummaryDoc>
material_id=MPID(mp-1120447)

Fields not requested:
['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal

把每一个ID整合成一个list

In [21]:
oxide_mp_ids = [e.material_id for e in entries]
print(oxide_mp_ids)

[MPID(mp-1120447), MPID(mp-1244933), MPID(mp-1244971), MPID(mp-1244990), MPID(mp-1245041), MPID(mp-1245242), MPID(mp-1403870), MPID(mp-644693), MPID(mp-676011), MPID(mp-988210), MPID(mp-1079297), MPID(mp-1204046), MPID(mp-1056579), MPID(mp-999200), MPID(mp-92), MPID(mp-571520), MPID(mp-1199894), MPID(mp-1202745), MPID(mp-1094057), MPID(mp-1268191), MPID(mp-1196961), MPID(mp-34), MPID(mp-10649), MPID(mp-1120747), MPID(mp-1203790), MPID(mp-165), MPID(mp-1201492), MPID(mp-168), MPID(mp-1200830), MPID(mp-971661), MPID(mp-971662), MPID(mp-27), MPID(mp-149), MPID(mp-16220), MPID(mp-1014212), MPID(mp-1072544), MPID(mp-1001113), MPID(mp-1095269), MPID(mp-1204627), MPID(mp-2351841), MPID(mp-1079649), MPID(mp-109), MPID(mp-1524805), MPID(mp-1073686), MPID(mp-1073695), MPID(mp-1073702), MPID(mp-1073720), MPID(mp-1073721), MPID(mp-1073730), MPID(mp-1073733), MPID(mp-1073734), MPID(mp-1073750), MPID(mp-1073785), MPID(mp-1073809), MPID(mp-1073816), MPID(mp-1073819), MPID(mp-1073822), MPID(mp-1073857

把所有的ID的list整合进all_mp_id.pkl文件用于后续下载；

其中含Si的MP_id有12673个

In [29]:
with open("all_mp_id_demo.pkl", "wb") as f:
    pickle.dump(oxide_mp_ids, f)
print(len(oxide_mp_ids))
print(oxide_mp_ids[132])
# with open("all_mp_id_demo.pkl", "rb") as f:
#     oxide_mp_ids = pickle.load(f)

12673
mp-615993


# STEP2：遍历all_mp_id.pkl文件，下载所有惯用晶胞CIF文件

In [75]:
from mp_api.client import MPRester
import re
from pymatgen.io.cif import CifWriter
import numpy as np
import pickle


class CifDownload:
    def __init__(self):
        #
        self.mp_rest_obj = MPRester("igRHy7zYOKzWD18jY76XtjbwEpRl6SoH")

    def to_cif(self, id, flag=True):
        if flag:
            id = 'mp-' + str(id)
            # id = str(id) - 'mp-'

        conventional_structure = self.mp_rest_obj.get_structure_by_material_id(id, conventional_unit_cell=True)
        # conventional_structure = self.mp_rest_obj.get_structure_by_material_id(id)

        cif_writer_obj = CifWriter(conventional_structure, write_magmoms=True)
        material = re.findall('Full\sFormula\s(.*?)\n', str(conventional_structure))[0].lstrip('(').rstrip(')')

        # name = material + "-" + str(id)
        name = str(id).strip('mp-')

        cif_writer_obj.write_file(r'conventional_cell/{}.cif'.format(name))


In [76]:
cif_download_obj = CifDownload()
# 12673

In [83]:
with open("all_mp_id_demo.pkl", "rb+") as f:
    all_mp_id_lst = pickle.load(f)
all_mp_id_lst.sort(key=lambda x: int(x.split("-")[-1]))
print(all_mp_id_lst)
print(len(all_mp_id_lst))

[MPID(mp-27), MPID(mp-34), MPID(mp-92), MPID(mp-109), MPID(mp-149), MPID(mp-165), MPID(mp-168), MPID(mp-186), MPID(mp-189), MPID(mp-206), MPID(mp-351), MPID(mp-355), MPID(mp-378), MPID(mp-381), MPID(mp-389), MPID(mp-449), MPID(mp-496), MPID(mp-507), MPID(mp-517), MPID(mp-531), MPID(mp-696), MPID(mp-729), MPID(mp-746), MPID(mp-752), MPID(mp-772), MPID(mp-775), MPID(mp-793), MPID(mp-795), MPID(mp-818), MPID(mp-828), MPID(mp-862), MPID(mp-871), MPID(mp-884), MPID(mp-893), MPID(mp-956), MPID(mp-976), MPID(mp-980), MPID(mp-988), MPID(mp-993), MPID(mp-1004), MPID(mp-1009), MPID(mp-1042), MPID(mp-1069), MPID(mp-1074), MPID(mp-1103), MPID(mp-1106), MPID(mp-1111), MPID(mp-1118), MPID(mp-1128), MPID(mp-1182), MPID(mp-1217), MPID(mp-1222), MPID(mp-1275), MPID(mp-1278), MPID(mp-1299), MPID(mp-1304), MPID(mp-1314), MPID(mp-1332), MPID(mp-1367), MPID(mp-1370), MPID(mp-1431), MPID(mp-1477), MPID(mp-1480), MPID(mp-1483), MPID(mp-1515), MPID(mp-1535), MPID(mp-1563), MPID(mp-1602), MPID(mp-1619), MPID(m

In [ ]:
lis_1000 = all_mp_id_lst[:1000]
for i in lis_1000:
        #num = int(i.split("-")[-1])
        #if num < 350000:
        #    continue
        print(i)
        # mp-350000 的后一个是 504097(裂开)
        try:
            cif_download_obj.to_cif(i, flag=False)
        except Exception as e:
            # ...
            print(e)

# STEP3：编写id_prop.csv文件

In [97]:
import csv

# 打开一个新的文件用于写入，如果文件不存在则创建
with open('conventional_cell/id_prop.csv', 'w', newline='') as csvfile:
    # 创建一个csv写入器
    writer = csv.writer(csvfile)

    # 写入标题行（可选）
    # writer.writerow(['ID', 'Property'])

    # 遍历列表，写入每一行数据
    for item in lis_1000:
        item = str(item).split('mp-')[1]
        print(item)
        writer.writerow([item, 1])

27
34
92
109
149
165
168
186
189
206
351
355
378
381
389
449
496
507
517
531
696
729
746
752
772
775
793
795
818
828
862
871
884
893
956
976
980
988
993
1004
1009
1042
1069
1074
1103
1106
1111
1118
1128
1182
1217
1222
1275
1278
1299
1304
1314
1332
1367
1370
1431
1477
1480
1483
1515
1535
1563
1602
1619
1620
1714
1727
1752
1818
1841
1860
1863
1898
1924
1989
2062
2075
2084
2105
2108
2145
2199
2245
2287
2291
2379
2402
2478
2488
2499
2517
2538
2567
2568
2582
2592
2620
2661
2699
2700
2755
2783
2798
2800
2836
2841
2844
2874
2877
2893
2895
2914
2917
2924
2932
2938
2948
2954
2961
2965
3018
3031
3035
3042
3078
3097
3106
3108
3128
3129
3147
3148
3160
3161
3166
3173
3179
3191
3193
3194
3213
3218
3224
3239
3262
3273
3275
3276
3278
3284
3292
3301
3328
3337
3347
3352
3382
3387
3388
3390
3406
3412
3437
3441
3456
3464
3470
3489
3507
3513
3520
3545
3547
3552
3555
3566
3585
3587
3595
3606
3618
3640
3651
3657
3677
3678
3698
3734
3775
3777
3779
3789
3826
3833
3854
3882
3895
3896
3907
3932
3938
3939
3949
39

# STEP4：